In [7]:
/**
 * Demonstrate how to generate and fit a multivaraite time series.
 *
 * @author Haksun Li
 */

%use s2

// construct a VAR model
fun model() : VARModel {
    // the intercept (constant) vector
    val mu: Vector = DenseVector(arrayOf(1.0, 2.0))
    // the AR coefficients (excluding the initial 1)
    val phi: Array<Matrix> = arrayOf<Matrix>(
        DenseMatrix(arrayOf(
            doubleArrayOf(0.2, 0.3),
            doubleArrayOf(0.0, 0.4))),
        DenseMatrix(arrayOf(
            doubleArrayOf(0.1, 0.2),
            doubleArrayOf(0.3, 0.1)))
    )
    // the white noise covariance matrix
    val sigma: Matrix = DenseMatrix(arrayOf(
        doubleArrayOf(0.2, 0.1), 
        doubleArrayOf(0.1, 0.2)))
    // construct a VAR model
    val model = VARModel(mu, phi, sigma)
    return model   
}

// generate a random sample set
fun mts() : MultivariateIntTimeTimeSeries {
    val sim = VARIMASim(model())
    sim.seed(1234567891L)
    val N = 5000 // number of samples
    val ts = arrayOfNulls<DoubleArray>(N)
    for (i in 0 until N) {
        ts[i] = sim.nextVector()
    }
    // a multivariate time series of random samples
    val mts: MultivariateIntTimeTimeSeries = MultivariateSimpleTimeSeries(*ts)
    return mts
}

// fit a VAR model
val fitted: VARModel = VARFit(mts(), 2)
println("mu: %s\n".format(fitted.mu()))
println("phi0: %s\n".format(fitted.AR(1)))
println("phi1: %s\n".format(fitted.AR(2)))

// convert a VAR model into a VECM model
val varx = VARXModel(
    fitted.mu(),
    fitted.phi(),
    DenseMatrix(2, 2).ZERO(),
    fitted.sigma()
)
val vecm = VECMLongrun(varx)
println("VECM")
println("the intercept (constant) vector:")
println(vecm.mu())
println("\nthe number of lags:")
println(vecm.p())
println("\nthe impact matrix:")
println(vecm.pi())
println("\nthe AR coefficients of the lagged differences:")
println(vecm.gamma(1))
println(vecm.gamma(2))
println("\nthe coefficients of the deterministic terms (excluding the intercept term):")
println(vecm.psi())
println("\nthe white noise covariance matrix:")
println(vecm.sigma())

mu: [1.074960, 1.980312] 

phi0: 2x2
	[,1] [,2] 
[1,] 0.214364, 0.275072, 
[2,] -0.005042, 0.408162, 

phi1: 2x2
	[,1] [,2] 
[1,] 0.106844, 0.197318, 
[2,] 0.293868, 0.106817, 

VECM
the intercept (constant) vector:
[1.074960, 1.980312] 

the number of lags:
3

the impact matrix:
2x2
	[,1] [,2] 
[1,] 0.321208, 0.472391, 
[2,] 0.288826, 0.514979, 

the AR coefficients of the lagged differences:
2x2
	[,1] [,2] 
[1,] 0.000000, 0.000000, 
[2,] 0.000000, 0.000000, 
2x2
	[,1] [,2] 
[1,] 0.214364, 0.275072, 
[2,] -0.005042, 0.408162, 

the coefficients of the deterministic terms (excluding the intercept term):
2x2
	[,1] [,2] 
[1,] 0.000000, 0.000000, 
[2,] 0.000000, 0.000000, 

the white noise covariance matrix:
2x2
	[,1] [,2] 
[1,] 1.000000, 0.000000, 
[2,] 0.000000, 1.000000, 
